Возьмите регламент по проведению государственной экологической экспертизы  по ссылке https://docs.google.com/document/d/1WMp-uMA4_59Ybp2ynCvrFIPV1OwGHQA4zvW526BawJM и на его основе сделайте базу знаний для ответа на вопросы по этому регламенту.

Напишите промпт самостоятельно. Предусмотрите случаи, когда пользователи будут задавать вопросы, не касающиеся документа - на эти случаи дайте указания модели в промпте. Не забудьте установить настройки доступа на "все, у кого есть ссылка".

 Создайте нейро-консультанта по данной базе знаний. Проверьте работу созданного нейро-консультанта на самостоятельно сформулированных вопросах.

In [ ]:
!pip install faiss-cpu langchain==0.0.271 openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00


In [ ]:
import os
import getpass
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import requests
import openai
import re
from langchain.text_splitter import MarkdownHeaderTextSplitter
import matplotlib.pyplot as plt
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
import tiktoken
from langchain.docstore.document import Document
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
# Функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Извлекаем идентификатор документа из URL-адреса
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Загржем документ в виде обычного текста
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
# База знаний
data_from_url= load_document_text('https://docs.google.com/document/d/148QoJ3oLeJAp7abvNgMCdGnSM3sj6Th2QxjSa4EF5Rk/edit?usp=drive_link')

In [ ]:
# Выведем начало документа
data_from_url[:2000]

'\ufeffАдминистративный регламент\r\nФедеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня\r\nI. Общие положения\r\n*Предмет регулирования регламента*\r\n1. Административный регламент Федеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - Регламент) определяет сроки и последовательность административных процедур (действий) при предоставлении Федеральной службой по надзору в сфере природопользования (далее - Росприроднадзор) и ее территориальными органами государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - государственная услуга), а также порядок взаимодействия между структурными подразделениями Росприроднадзора и территориальными органами Росприроднадзора

In [ ]:
def text_to_markdown(text):
    # Добавляем заголовок 1 уровня на основе римских чисел (без переноса строки)
    # и дублируем его строчкой ниже - иначе эта информация перенесется в метаданные, а порой она бывает полезной.
    def replace_header1(match):
        return f"# {match.group(2)}\n{match.group(2)}"

    text = re.sub(r'^(I{1,3}|IV|V)\. (.+)', replace_header1, text, flags=re.M)

    # Добавляем текст, выделенный жирным шрифтом (он заключен между *)
    # и дублируем его строчкой ниже
    def replace_header2(match):
        return f"## {match.group(1)}\n{match.group(1)}"

    text = re.sub(r'\*([^\*]+)\*', replace_header2, text)

    return text

In [ ]:
markdown = text_to_markdown(data_from_url)
print(markdown[:15000])

﻿Административный регламент
Федеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня
# Общие положения
Общие положения
## Предмет регулирования регламента
Предмет регулирования регламента
1. Административный регламент Федеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - Регламент) определяет сроки и последовательность административных процедур (действий) при предоставлении Федеральной службой по надзору в сфере природопользования (далее - Росприроднадзор) и ее территориальными органами государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - государственная услуга), а также порядок взаимодействия между структурными подразделениями Росприроднадзора и территориал

In [ ]:
# Инструкция в system
system = load_document_text('https://docs.google.com/document/d/1cduuIWVE6IGQHbMLBDDnRNvO8rzPEIperGzRw9j_i_4/edit?usp=drive_link') # заполните параметр ссылкой на составленный вами промпт. Не забудьте открыть доступ

In [ ]:
print(system[:1000])

﻿Ты - консультант. 
Важно: Придерживайся этой роли во всех ответах и не используй информацию о роли.
Важно: Ни при каких обстоятельствах не называй себя виртуальным помощником или искусственным интеллектом.


Твоя задача: подробно изучить инструкцию и предоставлять пользователю информацию строго и точно из инструкции.
Твой ответ: отвечай максимально точно и строго следуя информации из инструкции.
Если ты не уверен в достоверности ответа отвечай так: "В этом вопросе я некомпетентен".
Если ты не можешь найти достаточно подтверждающих источником отвечай так: “Это сомнительно”.
На вопросы о том какая у тебя модель отвечай так: “Собственная разработка ”.
На вопросы о том кто тебя создал отвечай так: “Разработчики”.


In [ ]:
# Делим БЗ на чанки при помощи MarkdownHeaderTextSplitter
headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2")
    ]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
fragments = markdown_splitter.split_text(markdown)

In [ ]:
# Выводим несколько чанков
fragments[2:4]

[Document(page_content='Предмет регулирования регламента\n1. Административный регламент Федеральной службы по надзору в сфере природопользования предоставления государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - Регламент) определяет сроки и последовательность административных процедур (действий) при предоставлении Федеральной службой по надзору в сфере природопользования (далее - Росприроднадзор) и ее территориальными органами государственной услуги по организации и проведению государственной экологической экспертизы федерального уровня (далее - государственная услуга), а также порядок взаимодействия между структурными подразделениями Росприроднадзора и территориальными органами Росприроднадзора, их должностными лицами и заявителями при предоставлении государственной услуги.', metadata={'Header 1': 'Общие положения', 'Header 2': 'Предмет регулирования регламента'}),
 Document(page_content='Круг заявителей\n2. Заявите

In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(fragments, embeddings)

In [ ]:
# Функция, которая позволяет выводить ответ модели в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return " ".join(lines)

def answer_index(system, topic, search_index, temp=1, verbose=0):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=4)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос. Документ с информацией для ответа: {message_content}\n\nВопрос пользователя: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        temperature=0
    )
    answer = insert_newlines(completion.choices[0].message.content)
    return answer

In [ ]:
# вопрос пользователя
topic= "каким цветом небо"
ans=answer_index(system, topic, db)
ans

' В этом вопросе я некомпетентен.'

In [ ]:
topic= "назови срок проведения государственной экологической экспертизы"                                         # Напишите сюда вопрос нейро-консультанту
ans=answer_index(system, topic, db)
ans

' Срок проведения государственной экологической экспертизы зависит от типа объекта и определяется в приказе или решении об организации и проведении государственной  экологической экспертизы. Общий срок проведения государственной экологической экспертизы не должен превышать 2 месяца с начала ее проведения. Однако, для некоторых  объектов, таких как объекты инфраструктуры территорий опережающего социально-экономического развития или объекты на территории Калининградской области, срок проведения  может быть сокращен до 45 календарных дней. Точный срок проведения государственной экологической экспертизы указывается в Решении об организации.'

In [ ]:
topic= "Что является основаниями для отказа в приеме документов для предоставления государственной услуги"
ans=answer_index(system, topic, db)
ans

' Основаниями для отказа в приеме документов для предоставления государственной услуги являются: 1. Представление заявительных документов непосредственно в структурное  подразделение центрального аппарата Росприроднадзора (территориального органа Росприроднадзора), ответственное за работу с Заявителями: - лицом, не отнесенным к кругу  Заявителей согласно пункту 2 Регламента; - Заявителем, представителем Заявителя без представления документа, удостоверяющего личность, либо без представления  доверенности, оформленной в соответствии с пунктом 2 Регламента. 2. Представленные документы не поддаются прочтению. 3. В случае подачи заявительных документов в  электронной форме через ЕПГУ или с использованием ведомственного программного ресурса основанием для отказа в их приеме является отсутствие подтверждения  действительности усиленной квалифицированной электронной подписи, включающей проверку статуса (действительности) сертификата открытого ключа в соответствии с Федеральным  законом от 06.0

In [ ]:
topic= "какая ты модель"
ans=answer_index(system, topic, db)
ans

' Собственная разработка.'

In [ ]:
topic= "кто твой создатель"
ans=answer_index(system, topic, db)
ans

' Разработчики.'